<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_metric_kernel_adam_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn matplotlib

In [ ]:
#!/usr/bin/env python3
"""
quantum_metric_kernel_adam.py

4‐bit parity classification via three methods:
1) Analytic CNOT cascade baseline (100% accuracy)
2) Variational QNN with 5‐fold cross‐validation
3) Quantum metric learning with centered‐alignment loss and Adam optimizer
4) Final SVM (precomputed kernel) 5‐fold CV
5) Kernel heatmap visualization
"""

import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Hyperparameters
# -----------------------------------------------------------------------------
n_bits     = 4        # number of input qubits / data bits
qnn_epochs = 30       # epochs for variational QNN
qnn_lr     = 0.4      # learning rate for variational QNN
metric_steps = 100    # steps for metric‐learning
metric_lr    = 0.2    # Adam learning rate for metric learning
svm_C        = 1.0    # SVM regularization
seed         = 42     # random seed

np.random.seed(seed)
pnp.random.seed(seed)

# -----------------------------------------------------------------------------
# 1) Generate parity dataset
# -----------------------------------------------------------------------------
X_bits   = np.array(list(itertools.product([0, 1], repeat=n_bits)))
y_bits   = np.sum(X_bits, axis=1) % 2          # 0=even, 1=odd
y_signed = 1 - 2 * y_bits                      # +1 for even, –1 for odd
X_feat   = X_bits.astype(float)                # for AngleEmbedding

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# -----------------------------------------------------------------------------
# 2) Analytic CNOT cascade baseline
# -----------------------------------------------------------------------------
dev_exact = qml.device("default.qubit", wires=n_bits + 1)

@qml.qnode(dev_exact)
def analytic_circuit(x):
    # prepare input bits
    for i, bit in enumerate(x):
        if bit:
            qml.PauliX(wires=i)
    # cascade CNOTs onto ancilla
    for i in range(n_bits):
        qml.CNOT(wires=[i, n_bits])
    # measure ancilla parity
    return qml.expval(qml.PauliZ(wires=n_bits))

preds_exact = [0 if analytic_circuit(x) > 0 else 1 for x in X_bits]
acc_exact   = accuracy_score(y_bits, preds_exact)
print(f"Analytic CNOT Cascade Accuracy: {acc_exact:.3f}")

# -----------------------------------------------------------------------------
# 3) Variational QNN baseline (5‐fold CV)
# -----------------------------------------------------------------------------
dev_qnn = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_qnn, interface="autograd")
def qnn_circuit(params, x):
    # angle‐encode input
    for i, bit in enumerate(x):
        if bit:
            qml.RY(np.pi, wires=i)
    # trainable rotations
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    # ladder entanglement
    for i in range(n_bits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.expval(qml.PauliZ(wires=n_bits - 1))

var_scores = []
for train_idx, test_idx in kf.split(X_bits, y_bits):
    # initialize parameters
    params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
    opt    = qml.GradientDescentOptimizer(stepsize=qnn_lr)

    # training loop
    for _ in range(qnn_epochs):
        def cost(p):
            preds = [qnn_circuit(p, X_bits[i]) for i in train_idx]
            return pnp.mean((pnp.array(preds) - y_signed[train_idx]) ** 2)
        params = opt.step(cost, params)

    # evaluate on held‐out fold
    preds_test = [qnn_circuit(params, X_bits[i]) for i in test_idx]
    bits_pred  = [0 if v > 0 else 1 for v in preds_test]
    var_scores.append(accuracy_score(y_bits[test_idx], bits_pred))

print(f"Variational QNN CV Accuracy: {np.mean(var_scores):.3f} ± {np.std(var_scores):.3f}")

# -----------------------------------------------------------------------------
# 4) Quantum metric learning with Adam
# -----------------------------------------------------------------------------
dev_feat = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_feat, interface="autograd")
def feature_map(params, x):
    # embed input angles
    qml.templates.AngleEmbedding(x, wires=range(n_bits))
    # layer 1: one Rot per qubit
    for i in range(n_bits):
        phi, theta, omega = params[3 * i : 3 * i + 3]
        qml.Rot(phi, theta, omega, wires=i)
    # ring entanglement
    for i in range(n_bits):
        qml.CNOT(wires=[i, (i + 1) % n_bits])
    # layer 2: RZ + Ry
    offset = 3 * n_bits
    for i in range(n_bits):
        qml.RZ(params[offset + i], wires=i)
        qml.RY(params[offset + n_bits + i], wires=i)
    return qml.state()

def build_kernel_matrix(params):
    # compute fidelity² Gram matrix
    states = pnp.stack([feature_map(params, x) for x in X_feat])
    re, im = pnp.real(states), pnp.imag(states)
    real_ov = re @ re.T + im @ im.T
    imag_ov = re @ im.T - im @ re.T
    return real_ov**2 + imag_ov**2

def centered_alignment(K):
    # classical centering + target‐kernel alignment
    N  = K.shape[0]
    H  = pnp.eye(N) - pnp.ones((N, N)) / N
    Kc = H @ K @ H
    T  = pnp.outer(y_signed, y_signed)
    num = pnp.sum(Kc * T)
    den = pnp.linalg.norm(Kc) * pnp.linalg.norm(T)
    return num / den

# initialize metric parameters: 3*n_bits + 2*n_bits
dim_params = 3 * n_bits + 2 * n_bits
params_k   = pnp.random.randn(dim_params, requires_grad=True) * 0.1
opt_metric = qml.AdamOptimizer(stepsize=metric_lr)

print("\nTraining feature map via centered alignment (Adam)")
for step in range(metric_steps):
    # loss = negative alignment
    def loss_fn(p):
        K = build_kernel_matrix(p)
        return -centered_alignment(K)

    params_k, loss_val = opt_metric.step_and_cost(loss_fn, params_k)
    if step % 20 == 0 or step == metric_steps - 1:
        align_val = centered_alignment(build_kernel_matrix(params_k))
        print(f" Step {step:3d}: alignment = {align_val:.4f}")

# -----------------------------------------------------------------------------
# 5) Final SVM CV on learned kernel
# -----------------------------------------------------------------------------
K_learned = np.array(build_kernel_matrix(params_k))
ker_scores = []

for tr, te in kf.split(K_learned, y_bits):
    svc = SVC(kernel="precomputed", C=svm_C)
    svc.fit(K_learned[np.ix_(tr, tr)], y_bits[tr])
    preds = svc.predict(K_learned[np.ix_(te, tr)])
    ker_scores.append(accuracy_score(y_bits[te], preds))

print(f"\nLearned Quantum‐Metric Kernel CV Accuracy: {np.mean(ker_scores):.3f} ± {np.std(ker_scores):.3f}")

# -----------------------------------------------------------------------------
# 6) Kernel heatmap
# -----------------------------------------------------------------------------
plt.figure(figsize=(4, 4))
plt.title("Learned Kernel Matrix |⟨ψ_i|ψ_j⟩|²")
plt.imshow(K_learned, cmap="viridis", origin="lower")
plt.colorbar(label="Kernel value")
plt.xlabel("Index j")
plt.ylabel("Index i")
plt.tight_layout()
plt.show()